In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from scipy.stats import f_oneway, kruskal
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
# Read CSV
sus_df = pd.read_csv('sus_results/sus.csv')

In [17]:
sus_df

,user_id,user_archetype,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10
0,0,a,4,2,4,2,4,1,3,2,4,4
1,1,b,1,4,3,1,3,4,5,2,5,1
2,2,a,3,3,2,2,4,1,3,3,5,1
3,3,b,4,2,4,1,4,2,4,2,4,2
4,4,a,4,2,4,1,4,1,5,2,4,2
5,5,b,2,3,2,1,4,2,4,2,3,2
6,6,a,4,2,4,2,4,2,4,2,4,2
7,7,b,3,2,4,1,4,2,4,2,3,2
8,8,a,4,1,4,2,4,2,4,2,4,2
9,9,b,4,2,4,2,3,3,4,2,3,2


In [4]:
# Transform for latex table

# Step 1: Melt the DataFrame
melted_df = sus_df.drop(columns=['user_id', 'user_archetype']).melt(var_name='Question', value_name='Value')

# Step 2: Group by Question and Value, and count occurrences
grouped = melted_df.groupby(['Question', 'Value']).size().unstack(fill_value=0)

# Step 3: Add the missing columns (1 to 5) if they don't exist
for i in range(1, 6):
    if i not in grouped.columns:
        grouped[i] = 0

# Step 4: Calculate the percentage
grouped_percentage = grouped.div(grouped.sum(axis=1), axis=0) * 100

# Step 5: Combine count and percentage
latex_result = grouped.astype(str) + " (" + grouped_percentage.round(0).astype(str) + "%)"

# Step 6: Reorder columns to ensure they're in the correct order
latex_result = latex_result[[1, 2, 3, 4, 5]]

# Step 7: Rename columns
latex_result.columns = ['Strongly disagree', 'Disagree', 'Neither agree nor disagree', 'Agree', 'Strongly agree']

# Step 9: Reset index and rename
#latex_result.reset_index(inplace=True)
#latex_result.rename(columns={'Question': 'TAM category'}, inplace=True)


In [21]:
latex_result

,Strongly disagree,Disagree,Neither agree nor disagree,Agree,Strongly agree
Question,,,,,
q1,1 (2.0%),4 (10.0%),12 (29.0%),23 (56.0%),1 (2.0%)
q10,7 (17.0%),29 (71.0%),4 (10.0%),1 (2.0%),0 (0.0%)
q2,6 (15.0%),27 (66.0%),5 (12.0%),3 (7.0%),0 (0.0%)
q3,0 (0.0%),2 (5.0%),4 (10.0%),31 (76.0%),4 (10.0%)
q4,13 (32.0%),24 (59.0%),3 (7.0%),1 (2.0%),0 (0.0%)
q5,0 (0.0%),4 (10.0%),6 (15.0%),26 (63.0%),5 (12.0%)
q6,5 (12.0%),27 (66.0%),6 (15.0%),2 (5.0%),1 (2.0%)
q7,0 (0.0%),0 (0.0%),3 (7.0%),28 (68.0%),10 (24.0%)
q8,7 (17.0%),27 (66.0%),5 (12.0%),2 (5.0%),0 (0.0%)


In [5]:
# Filter 
filtered_sus_df = sus_df.iloc[:, 2:]
# Select columns with even and odd indices
odd_index_columns = filtered_sus_df.iloc[:, ::2]
even_index_columns = filtered_sus_df.iloc[:, 1::2]
# Subtract 1 from odd_index_columns
odd_index_columns = odd_index_columns - 1
# Subtract from 5 for even_index_columns
even_index_columns = 5 - even_index_columns
# Combine
processed_sus_df = even_index_columns.join(odd_index_columns)
# Sum
processed_sus_df['score'] = processed_sus_df.sum(axis=1)*2.5
# Average
print(processed_sus_df['score'].mean())
# Std
print(processed_sus_df['score'].std())

72.3170731707317
8.79257932688197


In [4]:
processed_sus_df

,q2,q4,q6,q8,q10,q1,q3,q5,q7,q9,score
0,3,3,4,3,1,3,3,3,2,3,70.0
1,1,4,1,3,4,0,2,2,4,4,62.5
2,2,3,4,2,4,2,1,3,2,4,67.5
3,3,4,3,3,3,3,3,3,3,3,77.5
4,3,4,4,3,3,3,3,3,4,3,82.5
5,2,4,3,3,3,1,1,3,3,2,62.5
6,3,3,3,3,3,3,3,3,3,3,75.0
7,3,4,3,3,3,2,3,3,3,2,72.5
8,4,3,3,3,3,3,3,3,3,3,77.5
9,3,3,2,3,3,3,3,2,3,2,67.5


In [6]:
# Read CSV
demographics = pd.read_csv('demographic_results/demographics_processed.csv')
# Create df for demographic analysis
sus_demographics_df = pd.DataFrame(processed_sus_df[['score']]).join(sus_df['user_id'])
sus_demographics_df = sus_demographics_df.merge(demographics[['user_id', 'user_archetype', 'age_new', 'sex_new', 'medical_speciality_new', 'grade_new', 'ai_familiarity_new']])
# Imputation with Mode for categorical columns
sus_demographics_df['age_new'].fillna(sus_demographics_df['age_new'].mode()[0], inplace=True)
sus_demographics_df['sex_new'].fillna(sus_demographics_df['sex_new'].mode()[0], inplace=True)
sus_demographics_df['medical_speciality_new'].fillna(sus_demographics_df['medical_speciality_new'].mode()[0], inplace=True)
sus_demographics_df['grade_new'].fillna(sus_demographics_df['grade_new'].mode()[0], inplace=True)
sus_demographics_df['ai_familiarity_new'].fillna(sus_demographics_df['ai_familiarity_new'].mode()[0], inplace=True)
# Drop
sus_demographics_df.drop(columns='user_id', inplace=True)

In [7]:
sus_demographics_df

,score,user_archetype,age_new,sex_new,medical_speciality_new,grade_new,ai_familiarity_new
0,70.0,a,30s,Male,Pharmacist,Other,Slightly familiar
1,62.5,b,30s,Male,Infectious Diseases,Other,Slightly familiar
2,67.5,a,30s,Male,Infectious Diseases,Other,Slightly familiar
3,77.5,b,30s,Female,Infectious Diseases,Other,Not familiar
4,82.5,a,30s,Male,Other,Consultant,Slightly familiar
5,62.5,b,40s,Male,Other,Consultant,Moderately familiar
6,75.0,a,40s,Male,Pharmacist,Consultant,Slightly familiar
7,72.5,b,30s,Female,Pharmacist,Other,Moderately familiar
8,77.5,a,30s,Female,Pharmacist,Other,Slightly familiar
9,67.5,b,40s,Male,Microbiology,Consultant,Slightly familiar


In [12]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import f_oneway, kruskal, shapiro, levene
from statsmodels.stats.multitest import multipletests

def anover_kw_test_correction(df, column, correction_method='holm'):
    # Test for normality (Shapiro-Wilk test on residuals)
    model = smf.ols(f"{column} ~ user_archetype + age_new + sex_new + medical_speciality_new + grade_new + ai_familiarity_new", data=df).fit()
    shapiro_stat, shapiro_p_value = shapiro(model.resid)

    # Levene's test for homogeneity of variances across different variables
    levene_p_values = {}

    independent_vars = ['user_archetype', 'age_new', 'sex_new', 'medical_speciality_new', 'grade_new', 'ai_familiarity_new']
    
    for var in independent_vars:
        groups = [df[column][df[var] == group] for group in df[var].dropna().unique()]
        levene_stat, levene_p = levene(*groups)
        levene_p_values[var] = levene_p

    # Determine whether ANOVA or Kruskal-Wallis should be used
    if shapiro_p_value >= 0.05 and all(p >= 0.05 for p in levene_p_values.values()):
        anova_bool = True
    else:
        anova_bool = False

    print('\n')
    print(f"ANOVA Assumption Met: {anova_bool}")
    print('\n')

    # Collect results for correction
    results = []
    valid_p_values = []
    p_indices = []  # Keep track of indices where valid p-values exist
    
    # Perform Collective ANOVA
    if anova_bool:
        print('Collective ANOVA')
        model = smf.ols(f"{column} ~ user_archetype + age_new + sex_new + medical_speciality_new + grade_new + ai_familiarity_new", data=df).fit()
        aov_table = sm.stats.anova_lm(model, typ=2)

        for index, row in aov_table.iterrows():
            sum_sq, df_value, f_value, pr_f = row
            results.append({'variable': index, 'test': 'ANOVA', 'statistic': f_value, 'p_value': pr_f})
            if not np.isnan(pr_f):  # Only add valid p-values
                valid_p_values.append(pr_f)
                p_indices.append(len(results) - 1)

    # Function to perform ANOVA
    def perform_anova(df, group_var):
        groups = [df[column][df[group_var] == group] for group in df[group_var].dropna().unique()]
        stat, p_value = f_oneway(*groups)
        return stat, p_value

    # Function to perform Kruskal-Wallis H test
    def perform_kruskal(df, group_var):
        groups = [df[column][df[group_var] == group] for group in df[group_var].dropna().unique()]
        stat, p_value = kruskal(*groups)
        return stat, p_value

    print('\nIndependent Tests')
    for var in independent_vars:
        if anova_bool:
            stat, p_value = perform_anova(df, var)
            test_name = 'ANOVA'
        else:
            stat, p_value = perform_kruskal(df, var)
            test_name = 'Kruskal-Wallis'
        
        results.append({'variable': var, 'test': test_name, 'statistic': stat, 'p_value': p_value})
        if not np.isnan(p_value):  # Only add valid p-values
            valid_p_values.append(p_value)
            p_indices.append(len(results) - 1)

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Apply multiple comparisons correction
    if valid_p_values:  # Ensure there are valid p-values to correct
        reject, corrected_p_values, _, _ = multipletests(valid_p_values, alpha=0.05, method=correction_method)

        # Insert corrected p-values and rejection decision at correct indices
        for idx, corrected_p, reject_flag in zip(p_indices, corrected_p_values, reject):
            results_df.at[idx, 'p_corrected'] = corrected_p
            results_df.at[idx, 'reject'] = reject_flag

    # Fill missing corrected p-values with NaN
    results_df['p_corrected'].fillna(np.nan, inplace=True)
    results_df['reject'].fillna(False, inplace=True)

    # Print results
    for _, row in results_df.iterrows():
        print(f"{row['test']} for {row['variable']}: Statistic={row['statistic']:.4f}, p-value={row['p_value']:.4f}, Corrected p-value={row['p_corrected']:.4f}")
        if row['reject']:
            print(f"  -> The differences in {row['variable']} remain statistically significant after correction.\n")
        else:
            print(f"  -> The differences in {row['variable']} are no longer statistically significant after correction.\n")

    return results_df

In [13]:
anova_kw_results = anover_kw_test_correction(sus_demographics_df, 'score')



ANOVA Assumption Met: True


Collective ANOVA

Independent Tests
ANOVA for user_archetype: Statistic=2.5062, p-value=0.1250, Corrected p-value=1.0000
  -> The differences in user_archetype are no longer statistically significant after correction.

ANOVA for age_new: Statistic=1.8648, p-value=0.1457, Corrected p-value=1.0000
  -> The differences in age_new are no longer statistically significant after correction.

ANOVA for sex_new: Statistic=0.0960, p-value=0.7590, Corrected p-value=1.0000
  -> The differences in sex_new are no longer statistically significant after correction.

ANOVA for medical_speciality_new: Statistic=1.5597, p-value=0.2221, Corrected p-value=1.0000
  -> The differences in medical_speciality_new are no longer statistically significant after correction.

ANOVA for grade_new: Statistic=0.3548, p-value=0.5564, Corrected p-value=1.0000
  -> The differences in grade_new are no longer statistically significant after correction.

ANOVA for ai_familiarity_new: Statistic=

/tmp/ipykernel_12748/849359908.py:87: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  results_df.at[idx, 'reject'] = reject_flag


In [16]:
anova_kw_results[['reject']].value_counts()
anova_kw_results[['p_corrected']].value_counts()
anova_kw_results[anova_kw_results['p_corrected'] < 0.5]
anova_kw_results

reject
False     13
Name: count, dtype: int64

p_corrected
1.0            12
Name: count, dtype: int64

,variable,test,statistic,p_value,p_corrected,reject


,variable,test,statistic,p_value,p_corrected,reject
0,user_archetype,ANOVA,2.506228,0.125042,1.0,False
1,age_new,ANOVA,1.864787,0.145659,1.0,False
2,sex_new,ANOVA,0.096032,0.759023,1.0,False
3,medical_speciality_new,ANOVA,1.559716,0.222052,1.0,False
4,grade_new,ANOVA,0.354797,0.556371,1.0,False
5,ai_familiarity_new,ANOVA,1.262700,0.306923,1.0,False
6,Residual,ANOVA,NaN,NaN,NaN,False
7,user_archetype,ANOVA,1.696704,0.200359,1.0,False
8,age_new,ANOVA,1.125798,0.359701,1.0,False
9,sex_new,ANOVA,0.422464,0.519521,1.0,False
